In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import os
import random

def show_df_as_scrolling_table(df):
    # Puts the scrollbar next to the DataFrame
    # https://stackoverflow.com/questions/42724327/pandas-dataframe-table-vertical-scrollbars
    from IPython.core.display import HTML
    display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
                df.to_html() +
                "</div>"))


In [10]:
PARAGRAPH_FILE = "paragraph_data.xlsx"

if os.path.exists(PARAGRAPH_FILE):
    selected_articles = pd.read_excel(PARAGRAPH_FILE)
else:
    import psycopg2
    from sqlalchemy import create_engine, text

    import json


    with open("credentials.json", 'r') as fh:
        pmc_credentials = json.load(fh)

    connection_string = 'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'.format(**pmc_credentials)

    engine = create_engine(connection_string)
    Q = r"""
    with
    ji0 as (
        select *, substring(year from 1 for 4) y2 from journal_issue
        where year ~ '^\d\d\d\d'
        and substring(year from 1 for 4) = year
    )
    ,
    ji as (
        select *, y2::int year_int from ji0
    )
    ,
    pcounts as (
        select pmid, count(*) num_paragraphs from paragraph group by pmid
    )
    ,
    articles as (
        select distinct d.pmid
            from descriptor d
            join descriptor_detail dd on d.descriptor_id = dd.id
            join ji on d.pmid = ji.pmid
            join pcounts on pcounts.pmid = d.pmid
            where dd.name = 'Drug Therapy'
            and ji.year_int > 2000
            and pcounts.num_paragraphs > 10
        limit 300
    )
    select p.pmid, p.paragraph_number, p.section_path, p.text 
    from paragraph p join articles a on p.pmid = a.pmid
    where length(p.text) > 1
    order by pmid, paragraph_number;
    """

    selected_articles = pd.read_sql(Q, con=engine)
    selected_articles.to_excel(PARAGRAPH_FILE)


selected_articles
# 

,Unnamed: 0,pmid,paragraph_number,section_path,text
0,0,11953888,0,Title,How to calculate the dose of chemotherapy
1,1,11953888,1,Abstract,Body surface area-dosing does not account for ...
2,2,11953888,2,Abstract,"British Journal of Cancer (2002) 86, 1297–1302..."
3,3,11953888,3,Abstract,© 2002 Cancer Research UK
4,4,11953888,4,NaN,Despite the recent advances in anticancer trea...
...,...,...,...,...,...
11522,11522,32702853,39,Author contributions,Funding acquisition: Xu Zhou
11523,11523,32702853,40,Author contributions,Supervision: Weifeng Zhu
11524,11524,32702853,41,Author contributions,Methodology: Weifeng Zhu
11525,11525,32702853,42,Author contributions,"Writing – original draft: Shuqing Li, Jianrong..."


In [3]:
show_df_as_scrolling_table( selected_articles[ [ r'\usepackage' in x for x in selected_articles['text']] ] )

In [4]:
section_heading_data = []
for section_path_str in selected_articles['section_path'].values:
    if isinstance(section_path_str, str):
        section_path = section_path_str.split(' || ')
        for i in range(len(section_path)):
            my_text = str(section_path[i])
            if my_text == 'Title':
                my_level = 0
            else:
                my_level = i+1
            section_heading_data.append({'text': my_text, 'level': my_level})

headings_df = pd.DataFrame(section_heading_data)

In [6]:
heading_count = Counter(headings_df['text'])

headings_df['heading_count'] = [heading_count[h] for h in headings_df['text']]

show_df_as_scrolling_table(headings_df)

,text,level,heading_count
0,Title,0,300
1,Abstract,1,905
2,Abstract,1,905
3,Abstract,1,905
4,THE PROBLEM OF UNDERDOSING,1,5
5,THE PROBLEM OF UNDERDOSING,1,5
6,THE PROBLEM OF UNDERDOSING,1,5
7,THE PROBLEM OF UNDERDOSING,1,5
8,THE PROBLEM OF UNDERDOSING,1,5
9,THE SIGNIFICANCE OF UNDERDOSING,1,1


In [7]:
headings_set = dict( [ (row['text'], row['level']) for row in headings_df.to_dict(orient='records')] )
len(headings_set)  # 2816

unique_headings_df = pd.DataFrame([{'text': heading, 'level': level, 'is_heading':1} for (heading, level) in headings_set.items()])
np.max(unique_headings_df['level'])  # 4
unique_headings_df

,text,level,is_heading
0,Title,0,1
1,Abstract,1,1
2,THE PROBLEM OF UNDERDOSING,1,1
3,THE SIGNIFICANCE OF UNDERDOSING,1,1
4,PREVENTION OF UNDERDOSING,1,1
...,...,...,...
2741,Search methods for the identification of studies,2,1
2742,Managing missing data,3,1
2743,Data synthesis,3,1
2744,Assessment of the quality of evidence,3,1


In [8]:
body_text_df = pd.DataFrame([{'text': row['text'], 'level': 6, 'is_heading':0} for row in selected_articles.to_dict(orient='records')])
body_text_df

,text,level,is_heading
0,How to calculate the dose of chemotherapy,6,0
1,Body surface area-dosing does not account for ...,6,0
2,"British Journal of Cancer (2002) 86, 1297–1302...",6,0
3,© 2002 Cancer Research UK,6,0
4,Despite the recent advances in anticancer trea...,6,0
...,...,...,...
11522,Funding acquisition: Xu Zhou,6,0
11523,Supervision: Weifeng Zhu,6,0
11524,Methodology: Weifeng Zhu,6,0
11525,"Writing – original draft: Shuqing Li, Jianrong...",6,0


In [9]:
EXCEL_FILE = 'section_heading_data.xlsx'

all_df = pd.concat([unique_headings_df, body_text_df], axis=0)
all_df.to_excel(EXCEL_FILE)